In [22]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [241]:
#Load the data
df = pd.read_pickle("../data/sentence_labels")
input_df = df[['Sentence']].head().copy()
label_df = df[['Labels']].head().copy()

In [299]:
# Format the input
from keras.preprocessing.sequence import pad_sequences
import torch

# Use wordpiece tokenization
input_df['Tokenized'] = input_df[['Sentence']].apply(lambda x: tokenizer.tokenize(x[0]), axis=1)

# Replace words with integers, add the special [CLS] and [SEP] tokens
input_df['Integerized'] = input_df[['Tokenized']].apply(lambda x: tokenizer.encode(x[0], add_special_tokens=True), axis=1)

# Pad and truncate all sentences so they are the same length
length = 50
input_df['Padded'] = input_df[['Integerized']].apply(lambda x: pad_sequences(x, maxlen=length, dtype="long", truncating="post", padding="post"), axis=1)

# The model only accepts Tensor, so convert the Padded Input
input_df['Input_Tensor'] = input_df[['Padded']].apply(lambda x: torch.tensor(x[0]), axis=1)

# Add attention mask. Attention is 0 for padding, else 1
input_df['Attention_Mask'] = input_df[['Padded']].apply(lambda x: (x[0] != 0).astype(int), axis=1)

# The model only accepts Tensor, so convert the Mask
input_df['Attention_Tensor'] = input_df[['Attention_Mask']].apply(lambda x: torch.tensor(x[0]), axis=1)


def check_sentence(index):
    for column in input_df:
        print(column)
        print(input_df[[column]].iloc[index][0])
        print()
    
check_sentence(0)

Sentence
I sin första reaktion på Sovjetledarens varningar deklarerade Litauens president Vytautas Landsbergis att " nu avvisar Gorbatjov vår utsträckta hand med extremt skarpa och hämndlystna ord " .

Tokenized
['I', 'sin', 'f', '##ö', '##rst', '##a', 're', '##ak', '##tion', 'p', '##å', 'So', '##v', '##jet', '##led', '##are', '##ns', 'var', '##ning', '##ar', 'de', '##kla', '##rera', '##de', 'Li', '##ta', '##uen', '##s', 'president', 'V', '##yt', '##aut', '##as', 'Lands', '##berg', '##is', 'at', '##t', '"', 'n', '##u', 'a', '##v', '##vis', '##ar', 'Go', '##rb', '##at', '##jo', '##v', 'v', '##å', '##r', 'u', '##ts', '##tr', '##ä', '##ck', '##ta', 'hand', 'me', '##d', 'ex', '##tre', '##m', '##t', 's', '##kar', '##pa', 'o', '##ch', 'h', '##ä', '##m', '##nd', '##ly', '##st', '##na', 'or', '##d', '"', '.']

Integerized
[101, 146, 11850, 175, 19593, 9731, 1161, 1231, 3715, 2116, 185, 16835, 1573, 1964, 18836, 2433, 8836, 2316, 15661, 3381, 1813, 1260, 27752, 25508, 2007, 5255, 1777, 23404, 1

In [303]:
print(x)

tensor([[  101,   146, 11850,   175, 19593,  9731,  1161,  1231,  3715,  2116,
           185, 16835,  1573,  1964, 18836,  2433,  8836,  2316, 15661,  3381,
          1813,  1260, 27752, 25508,  2007,  5255,  1777, 23404,  1116,  2084,
           159, 25669, 24723,  2225, 17854,  2953,  1548,  1120,  1204,   107,
           183,  1358,   170,  1964,  9356,  1813,  3414, 26281,  2980,  5077]])


In [304]:
from transformers import DistilBertModel, DistilBertConfig
from transformers import DistilBertTokenizer, DistilBertModel
model = DistilBertModel.from_pretrained('distilbert-base-cased')

index = 0
x = input_df[['Padded']].iloc[index][0]
#The model only accepts tensors
x = torch.tensor(x)


y = input_df[['Attention_Tensor']].iloc[index][0]


embeddings = model.forward(input_ids=x,
    attention_mask=y,
    head_mask=None)

embeddings[0]

tensor([[[-0.4820,  0.0119, -0.2879,  ...,  0.0226,  0.1837, -0.0811],
         [-0.6348, -0.1782, -0.2193,  ...,  0.0683,  0.1993, -0.1413],
         [-0.5890, -0.2142, -0.3412,  ...,  0.1561,  0.1721, -0.2455],
         ...,
         [-0.1823, -0.3673, -0.3925,  ...,  0.2181, -0.3688, -0.6873],
         [-0.4502, -0.4721, -0.6138,  ..., -0.3842, -0.3976, -0.1746],
         [-0.4951, -0.3540, -0.2674,  ...,  0.1719, -0.0565, -0.0611]]],
       grad_fn=<NativeLayerNormBackward>)

In [21]:
from sklearn.model_selection import train_test_split

# Split into training and validation sets.

#Albert takes both the sentence tokens (as ids), the labels (as ids) and attention masks as input, so we split those as well.
r=random_state = 2018
t=test_size = 0.1

tr_token, val_token = train_test_split(df['Token_ID'].to_list(),random_state=r, test_size=t)
tr_class, val_class = train_test_split(df['Entity_ID'].to_list(),random_state=r, test_size=t)
tr_masks, val_masks = train_test_split(df['Attention_Mask'].to_list(),random_state=r, test_size=t)

KeyError: 'Token_ID'

In [10]:
import torch
#Convert to tensors

text_enc = tokenizer.encode(text, add_special_tokens=True)
mask_enc = tokenizer.encode(mask, add_special_tokens=True)
tags_enc = tokenizer.encode(tags, add_special_tokens=True)

print(text_enc)
print(mask_enc)
print(tags_enc)

text_enc_tensor = torch.tensor(text_enc).unsqueeze(0)
mask_enc_tensor = torch.tensor(mask_enc).unsqueeze(0)
tags_enc_tensor = torch.tensor(tags_enc).unsqueeze(0)



tr_inputs = torch.tensor(tr_token)
val_inputs = torch.tensor(val_token)
tr_tags = torch.tensor(tr_class)
val_tags = torch.tensor(val_class)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [11]:
#Convert it all into a tensordataset 
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#Set batch size
bs = 1

train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [12]:
#Load model
from transformers import DistilBertModel, DistilBertConfig
model = DistilBertModel.from_pretrained('distilbert-base-cased')

In [13]:
#Pass the data throug it to get embeddings
embeddings = model.forward(input_ids=tr_inputs,
    attention_mask=tr_masks,
    head_mask=None)

RuntimeError: index out of range: Tried to access index 41611 out of table with 28995 rows. at /pytorch/aten/src/TH/generic/THTensorEvenMoreMath.cpp:418